# Automated Term Extraction

In [12]:
from pyate import combo_basic
import googletrans
import feedparser
from urllib.request import urlopen
from bs4 import BeautifulSoup
from gensim.summarization import keywords
import gensim



def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

In [2]:
url_expansion = 'https://e00-expansion.uecdn.es/rss/empresasbanca.xml'
expansion = feedparser.parse(url_expansion)
expansion_entries = expansion['entries']

print('Total Entradas Expansión:',len(expansion_entries))

Total Entradas Expansión: 35


In [33]:
string_es = get_only_text(expansion_entries[5]['id'])[0]

In [34]:
string_es

'En términos de resultados sería un impacto del 1% sobre el beneficio bruto previsto por el banco de inversión para el sector correspondiente al ejercicio 2022. Barclays limita a una horquilla entre 700 y 1.000 millones de euros el coste para la banca del dictamen emitido por el Tribunal de Justicia de la Unión Europea (TJUE) donde establece la obligación de devolver las cantidades cobradas en concepto de gastos de hipoteca que estén incluidas en cláusulas declaradas abusivas, salvo que el derecho nacional establezca lo contrario. En términos de resultados sería un impacto del 1% sobre el beneficio bruto previsto por el banco de inversión para el sector correspondiente al ejercicio 2022, ya que los litigios se prolongarán en el tiempo al tener los tribunales que pronunciarse caso por caso, tal y como prevé en un informe publicado este lunes.  Para efectuar el cálculo, el banco de inversión toma la hipótesis de que recurrirán el 50% de los clientes hipotecados cinco años antes de entrar

In [35]:
string = googletrans.Translator().translate(string_es,dest='en').text
print(combo_basic(string).sort_values(ascending=False).head(5))

gross profit       2.079442
investment bank    2.079442
new mortgage       1.443147
acts tax           1.443147
legal acts         1.443147
dtype: float64


In [36]:
import spacy
from pyate.term_extraction_pipeline import TermExtractionPipeline

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(TermExtractionPipeline())
doc = nlp(string)
print('### ENG ###')
print(doc._.combo_basic.sort_values(ascending=False).head(5))
doc._.combo_basic.index = [googletrans.Translator().translate(i,dest='es').text for i in doc._.combo_basic.index]
print('### ESP ###')
print(doc._.combo_basic.sort_values(ascending=False).head(5))

### ENG ###
gross profit       2.079442
investment bank    2.079442
mortgage law       1.443147
new mortgage       1.443147
acts tax           1.443147
dtype: float64
### ESP ###
beneficio bruto         2.079442
Banco de inversiones    2.079442
ley hipotecaria         1.443147
nueva hipoteca          1.443147
actúa impuesto          1.443147
dtype: float64


In [37]:
keywords(string).split('\n')

['mortgage',
 'mortgaged',
 'law',
 'associations',
 'associated',
 'profit',
 'bank',
 'banks',
 'banking',
 'barclays',
 'year',
 'years',
 'fees',
 'actually',
 'explained',
 'requires',
 'declared',
 'cjeu',
 'union',
 'legal acts tax']

In [38]:
gensim.summarization.keywords(string,split=True,scores=True,words=10)

[('mortgage', 0.2543473015361933),
 ('mortgaged', 0.2543473015361933),
 ('law', 0.20713738495718073),
 ('profit', 0.19455766881763562),
 ('associations', 0.19455766881763545),
 ('associated', 0.19455766881763545),
 ('bank', 0.16263987854102985),
 ('banks', 0.16263987854102985),
 ('banking', 0.16263987854102985),
 ('barclays', 0.16090141482775516),
 ('year', 0.15869003185265254),
 ('years', 0.15869003185265254),
 ('explained', 0.14797343825566717),
 ('actually', 0.14797343825566703),
 ('fees', 0.1479734382556669)]